# Load libraries

In [4]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Flatten
from tensorflow.keras.layers import Embedding
from keras.models import Sequential
from tensorflow.keras import layers, models, losses, optimizers,callbacks

from keras.layers import Dense,LSTM
from keras.utils import pad_sequences


import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
stopwords = stopwords.words('english')

# load preprocessed data

In [8]:
df = pd.read_csv('preprocessed_data.csv')
df.shape

(158353, 6)

In [9]:
df['clean_lyrics'] = df['clean_lyrics'].astype(str)

# Create X and y feature

In [10]:
df = df.sample(frac=1)

X = df['clean_lyrics']

y = df['label']

# Vectorize X

In [11]:
# Limiting our tokenizers vocab size
max_words = 10000
 
    
# create the tokenizer
tokenizer = Tokenizer(num_words=max_words)


# Fit the tokenizer
tokenizer.fit_on_texts(X)


# Create the sequences for each sentence, basically turning each word into its index position
sequences = tokenizer.texts_to_sequences(X)


index_word = tokenizer.index_word


# # Limiting our sequencer to only include 300 words
max_length = 300


# # Convert the sequences to all be the same length of 300
X = pad_sequences(sequences, maxlen=max_length, padding='post')
print(X.shape)

(158353, 300)


# Split train and test sets

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)
X_train.shape,y_train.shape

((126682, 300), (126682,))

# Building LSTM nueral net 

In [7]:
# This creates the Neural Network
model = Sequential() 

# This embedding layer basically will automatically create the word2vec vectors based on your text data.
model.add( Embedding(max_words, 32, input_length=max_length) ) 

model.add(LSTM(50,return_sequences=True,dropout =0.2))
model.add(LSTM(50,dropout =0.2))
model.add(Dense(1,kernel_initializer='normal',  activation='linear'))
optimizer = optimizers.Adam(lr=0.003)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse']) 

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 32)           320000    
                                                                 
 lstm (LSTM)                 (None, 300, 50)           16600     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 356,851
Trainable params: 356,851
Non-trainable params: 0
_________________________________________________________________


c:\users\stephen williams\downloads\nlp-extra-credit-main\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


# Training

In [8]:
callback = callbacks.EarlyStopping(monitor='val_mse',patience = 2,restore_best_weights=True)
hist = model.fit(X_train, y_train, 
                 validation_split=0.2,
                 epochs=15, batch_size=20,callbacks=[callback])

Epoch 1/15


c:\users\stephen williams\downloads\nlp-extra-credit-main\env\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


5068/5068 [==============================] - 548s 107ms/step - loss: 0.0618 - mse: 0.0618 - val_loss: 0.0554 - val_mse: 0.0554
Epoch 2/15
5068/5068 [==============================] - 577s 114ms/step - loss: 0.0535 - mse: 0.0535 - val_loss: 0.0518 - val_mse: 0.0518
Epoch 3/15
5068/5068 [==============================] - 587s 116ms/step - loss: 0.0507 - mse: 0.0507 - val_loss: 0.0507 - val_mse: 0.0507
Epoch 4/15
5068/5068 [==============================] - 582s 115ms/step - loss: 0.0486 - mse: 0.0486 - val_loss: 0.0506 - val_mse: 0.0506
Epoch 5/15
5068/5068 [==============================] - 562s 111ms/step - loss: 0.0468 - mse: 0.0468 - val_loss: 0.0506 - val_mse: 0.0506
Epoch 6/15
5068/5068 [==============================] - 524s 103ms/step - loss: 0.0450 - mse: 0.0450 - val_loss: 0.0501 - val_mse: 0.0501
Epoch 7/15
5068/5068 [==============================] - 541s 107ms/step - loss: 0.0437 - mse: 0.0437 - val_loss: 0.0509 - val_mse: 0.0509
Epoch 8/15
5068/5068 [=======================

In [ ]:
model.save('LSTM_Valence_model')

In [5]:
model = keras.models.load_model('LSTM_Valence_model')


# Testing

In [14]:
mse= model.evaluate(X_test, y_test, verbose=0)[1]

print('Test mse with stacked LSTM:', mse)

Test mse with stacked LSTM: 0.05180038884282112
